In [1]:
import pickle as pkl
import numpy
import pandas as pd
import autograd.numpy as np
from autograd import grad 
# from rl_functions import off_policy_per_decision_weighted_doubly_robust as WDR
from rl_functions import value_iteration
from rl_functions import learn_rewards_function
from rl_functions import learn_transition_function
from rl_functions import turn_policy_to_stochastic_policy

In [2]:
train_set = pd.read_csv('../../data/train_scaled_encoded.csv')
test_set = pd.read_csv('../../data/test_scaled_encoded.csv')

In [3]:
train_input = pd.read_csv('../../data/train_input_features.csv')
test_input = pd.read_csv('../../data/test_input_features.csv')

In [4]:
"""off_policy_per_decision_weighted_doubly_robust(
    states_sequence, actions_sequence, rewards_sequence, fence_posts, gamma,
    pi_evaluation, pi_behavior, V = None, Q = None, num_of_states = None, num_of_actions = None )"""

'off_policy_per_decision_weighted_doubly_robust(\n    states_sequence, actions_sequence, rewards_sequence, fence_posts, gamma,\n    pi_evaluation, pi_behavior, V = None, Q = None, num_of_states = None, num_of_actions = None )'

In [6]:
train_state_list = pkl.load(open('../../data/classify_state/train_states.pkl', 'rb'), encoding='latin1')
test_state_list = pkl.load(open('../../data/classify_state/test_states.pkl', 'rb'), encoding='latin1')

In [7]:
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv, vaso)] = count
        count += 1

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
def get_experts_action_probs(w, f):
    p = sigmoid(np.sum(f * w, axis=1, keepdims=True))
    return p

In [10]:
expert1, expert2 = np.array(actions).reshape(-1,1), np.array(actions).reshape(-1,1)
#np.random.randint(0, 25, size=(train_input.shape[0], 1)), np.random.randint(0, 25, size=(train_input.shape[0], 1))

NameError: name 'actions' is not defined

In [11]:
encoded_expert1 = np.zeros((expert1.shape[0], 25))
encoded_expert2 = np.zeros((expert2.shape[0], 25))
for i, e1a in enumerate(expert1):
    encoded_expert1[i, e1a[0]] = 1
    encoded_expert2[i, expert2[i][0]] = 1

NameError: name 'expert1' is not defined

In [47]:
rewards = train_input['intermediate_reward'].values

In [321]:
w = np.random.rand(1, 3)

In [40]:
probs = get_experts_action_probs(w, train_input.values[:,:-1])

In [118]:
actions = []
vasos = train_set['vaso_input'].values
ivs = train_set['iv_input'].values
for i, iv in enumerate(ivs):
    actions += [ action_map[ (vasos[i], iv) ] ]

In [131]:
bloc = train_input['num_bloc'].values
trasition = np.zeros((750, 25, 750))
for i, s in enumerate(train_state_list):
    if i == train_state_list.shape[0] - 1:
        break
    if bloc[i + 1] == 1:
        continue
    else:
        trasition[s, actions[i], train_state_list[i + 1]] += 1

for s in range(750):
    trasition[s] /= np.sum(trasition[s], axis=1, keepdims=True)
trasition = np.nan_to_num(trasition)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [134]:
reward_table = np.zeros((750, 25, 750))
count = np.zeros((750, 25, 750))
for i, s in enumerate(train_state_list):
    if i == train_state_list.shape[0] - 1:
        break
    if bloc[i + 1] == 1:
        continue
    else:
        reward_table[s, actions[i], train_state_list[i + 1]] += rewards[i]
        count[s, actions[i], train_state_list[i + 1]] += 1
reward_table = np.nan_to_num(reward_table / count)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [186]:
t_table = np.zeros((750, 25))

In [187]:
for i, state in enumerate(train_state_list):
    t_table[state, expert1[i]] += probs[i]
    t_table[state, expert2[i]] += (1 - probs[i])

In [189]:
t_table = t_table / np.sum(t_table, axis=1, keepdims=True)

In [141]:
def policy_eval(MDP, prob_e, gamma=.99):
	# solve by value iteration
	(transition_matrix, reward_table) = MDP
	# print MDP
	V = np.zeros((transition_matrix.shape[0]))
	# compute V table
	while 1:
		
		delta = 0.
		
		for s in range(transition_matrix.shape[0]):
			v = np.sum(prob_e[s] * np.sum(transition_matrix[s] * (reward_table[s] + gamma * V), axis = 1))
			delta = max(delta, abs(v - V[s]))
			V[s] = v

		if delta < 0.001:
			break
			
	# build Q_table
	Q = np.zeros((transition_matrix.shape[0], transition_matrix.shape[1]))
	for s in range(transition_matrix.shape[0]):
		Q[s] = np.sum(transition_matrix[s] * (reward_table[s] + gamma * V), axis = 1)
	return V, Q

In [153]:
V, Q = policy_eval((trasition, reward_table), t_table)

In [218]:
pi_b = np.zeros((750, 25))
for i, s in enumerate(train_state_list):
    if i == train_state_list.shape[0] - 1:
        break
    pi_b[s, actions[i]] += 1
pi_b = pi_b / np.sum(pi_b, axis=1, keepdims=True)

In [164]:
fence_posts = []
for i, idx in enumerate(bloc):
    if idx == 1:
        fence_posts += [ i ]

In [181]:
moe_actions = []
for i, prob in enumerate(probs):
    if prob >= 0.5:
        moe_actions += [ expert1[i][0] ]
    else:
        moe_actions += [ expert2[i][0] ]

In [222]:
def WDR(
    states_sequence, actions_sequence, rewards_sequence, fence_posts, gamma,
    pi_evaluation, pi_behavior, V = None, Q = None, num_of_states = None, num_of_actions = None ):

    num_of_trials = len( fence_posts )
    individual_trial_estimators = []
    pi_evaluation = turn_policy_to_stochastic_policy( \
        pi_evaluation, num_of_states = num_of_states, num_of_actions = num_of_actions )
    pi_behavior = turn_policy_to_stochastic_policy( \
        pi_behavior, num_of_states = num_of_states, num_of_actions = num_of_actions )
    # estimate V and Q if they are not passed as parameters
    if V is None or Q is None:
        # TODO : add part which calculate R and T from data if they are not given
        V, Q = policy_evaluation( T , R , pi_evaluation , gamma )
    # calculate the doubly robust estimator of the policy
    fence_posts_with_length_appended = fence_posts + [ len( states_sequence ) ]
    single_patient_sequences_length = [ fence_posts_with_length_appended[i+1] - \
        fence_posts_with_length_appended[i] for i in range(len(fence_posts)) ]
    length_of_longest_patient_sequence = max( single_patient_sequences_length )
    rho_array = np.nan * np.zeros( ( num_of_trials, length_of_longest_patient_sequence ) )
#    rho_array = np.ones( ( num_of_trials, length_of_longest_patient_sequence ) )
    for trial_i in range( num_of_trials ):
        rho = 1
        if trial_i < num_of_trials - 1:
            steps_in_trial = fence_posts[ trial_i+1 ] -  fence_posts[ trial_i ]
        else:
            steps_in_trial = len( states_sequence) - fence_posts[-1]
        t_within_trial = 0
        for t in range(
                fence_posts[ trial_i], fence_posts[ trial_i] + steps_in_trial ):
            if(pi_behavior[ states_sequence[ t], actions_sequence[ t]]==0):
                print(states_sequence[ t], actions_sequence[ t])
            rho *= pi_evaluation[ states_sequence[ t], actions_sequence[ t]] / \
                pi_behavior[ states_sequence[ t], actions_sequence[ t]]
            rho_array[ trial_i, t_within_trial ] = rho
            t_within_trial += 1
        rho_array[ trial_i, t_within_trial: ] = rho
    weights_normalization = np.sum( rho_array, axis = 0 )
    for trial_i in range( num_of_trials ):
        current_trial_estimator = 0
        rho = 1
        w = 1 / num_of_trials
        discount = 1/gamma
        if trial_i < num_of_trials - 1:
            steps_in_trial = fence_posts[ trial_i+1 ] -  fence_posts[ trial_i ]
        else:
            steps_in_trial = len( states_sequence) - fence_posts[-1]
        t_within_trial = 0
        for t in range(
                fence_posts[ trial_i], fence_posts[ trial_i] + steps_in_trial ):
            previous_w = w
            rho *= pi_evaluation[ states_sequence[ t], actions_sequence[ t]] / \
                pi_behavior[ states_sequence[ t], actions_sequence[ t]]
            w = rho / weights_normalization[ t_within_trial ]
            discount *= gamma
            current_trial_estimator += w * discount * rewards_sequence[ t ] - \
                discount * ( w * Q[ states_sequence[ t ], actions_sequence[ t ] ] - \
                             previous_w * V[ states_sequence[ t ] ] )
            t_within_trial += 1
        individual_trial_estimators += [ current_trial_estimator ]
    estimator = np.sum( individual_trial_estimators )

    return estimator

In [285]:
def IS(
    states_sequence, actions_sequence, rewards_sequence, fence_posts, gamma,
    pi_evaluation, pi_behavior, num_of_states = None, num_of_actions = None ):

    num_of_trials = len( fence_posts )
    individual_trial_estimators = 0
    pi_evaluation = turn_policy_to_stochastic_policy( \
        pi_evaluation, num_of_states = num_of_states, num_of_actions = num_of_actions )
    pi_behavior = turn_policy_to_stochastic_policy( \
        pi_behavior, num_of_states = num_of_states, num_of_actions = num_of_actions )
    for trial_i in range( num_of_trials ):
        rho = 1
        discount = 1/gamma
        trial_return = 0
        if trial_i < num_of_trials - 1:
            steps_in_trial = fence_posts[ trial_i+1 ] -  fence_posts[ trial_i ]
        else:
            steps_in_trial = len( states_sequence) - fence_posts[-1]
        for t in range(
                fence_posts[ trial_i], fence_posts[ trial_i] + steps_in_trial ):
            rho *= pi_evaluation[ t, actions_sequence[ t]] / \
                pi_behavior[ states_sequence[ t], actions_sequence[ t]]
            discount *= gamma
            trial_return += discount * rewards_sequence[ t ]
        individual_trial_estimators += trial_return * rho 
   
    
    return individual_trial_estimators/num_of_trials

In [309]:
WDR(train_state_list, actions, rewards, fence_posts, 0.99, t_table, pi_b, V=V, Q=Q)

-70.631292302678901

In [324]:
def loss(w):
    
    probs = get_experts_action_probs(w, train_input.values[:,:-1])
    pi_e = probs * encoded_expert1 + (1 - probs) * encoded_expert2
    
    print (pi_e[0])
    
    return IS(train_state_list, actions, rewards, fence_posts, 0.99, pi_e, pi_b)

In [316]:
grad_loss = grad(loss)
for i in range(3):
    gradient = grad_loss(w)
    print (gradient)
    print (w)
    w -= 0.05 * gradient
    print ('loss:', loss(w))

[[ -2.84277585e+36   3.46442327e+35  -2.42336258e+34]]
[[ 0.68092459  0.98688324  0.45203845]]
loss: -3.52625434905e+36


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:73: RuntimeWarning: invalid value encountered in multiply
  defvjp(anp.exp,    lambda ans, x : lambda g: ans * g)


[[ nan  nan  nan]]
[[  1.42138793e+35  -1.73221163e+34   1.21168129e+33]]
loss: nan


KeyboardInterrupt: 

In [325]:
loss(w)

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


-3.5240917323904589e+36